In [1]:
import os
import fnmatch
import shutil
import xml.etree.ElementTree as ET
import glob
from xml.dom import minidom
import re
import subprocess

In [2]:
def replaceAll(search, replace, reg = True, flags = re.I | re.M, no_titel = False, files = []):
    
    if not files:
        files = glob.glob("new//OEBPS/*.xhtml")
    
    for file in files:
             
        if file == "new//OEBPS/toc.xhtml" and not files:
            continue
        if no_titel:
            if (file == "new//OEBPS/toc.xhtml") or (file == "new//OEBPS/cover.xhtml") or file == "new//OEBPS/titel_und_titelei.xhtml" or file == "new//OEBPS/impressum.xhtml":
                print(file)
                continue
            
        content_new = ""
        with open (file, 'r' ) as f:
            content = f.read()
            
            if reg:
                content_new = re.sub(search, replace, content, flags = flags)
            else:
                content_new = content.replace(search, replace)

        with open (file, 'w' ) as f:
            f.write(content_new)

In [3]:
old_epub = "old.epub"
new_epub = "new.epub"
auflage = "5"
datum = "21.10.2020"


In [4]:
d = datum.split(".")

date = d[2]+"-"+d[1]+"-"+d[0]+"T12:15:51+00:00"

In [5]:
old_zip = os.path.splitext(old_epub)[0] + ".zip"
new_zip = os.path.splitext(new_epub)[0] + ".zip"

shutil.copy(new_epub, new_zip)
shutil.copy(old_epub, old_zip)

# extract the epubs
os.system('unzip '+old_zip+' -d old')
os.system('unzip '+new_zip+' -d new')

os.remove(old_zip)
os.remove(new_zip)

In [6]:
# alle css dateien ersetzen
old_css_files = os.listdir('old//OEBPS//css/')
new_css_files = os.listdir('new//OEBPS//css/')

In [7]:
# calibre bookmarks entfernen
if os.path.exists('new//META-INF//calibre_bookmarks.txt'):
    os.remove('new//META-INF//calibre_bookmarks.txt')

In [8]:
shutil.rmtree('new//OEBPS//css/')
shutil.copytree('old//OEBPS//css/', 'new//OEBPS//css/')

# css dateien im header löschen
for new_css_file in new_css_files:
    if new_css_file not in old_css_files:
        find = '<link href="css/'+new_css_file+'" rel="stylesheet" type="text/css" />'
        replaceAll(find,"", reg=False)

In [9]:
# alle fonts ersetzen

shutil.rmtree('new//OEBPS//font/')
shutil.copytree('old//OEBPS//font/', 'new//OEBPS//font/')

# encryption töten
os.remove('new//META-INF//encryption.xml')

In [10]:
# content.opf rübercopieren
shutil.copy('old//OEBPS//content.opf', 'new//OEBPS//')

# titel_und_titelei.xhtml und toc rübercopieren
shutil.copy('old//OEBPS//titel_und_titelei.xhtml', 'new//OEBPS//')
shutil.copy('old//OEBPS//toc.ncx', 'new//OEBPS//')
shutil.copy('old//OEBPS//toc.xhtml', 'new//OEBPS//')

shutil.copy('old//OEBPS//impressum.xhtml', 'new//OEBPS//')

'new//OEBPS//impressum.xhtml'

In [11]:
# images folder erstetzen. titelbild austauschen
shutil.copy('new//OEBPS//image//1.png', 'new//OEBPS//')

shutil.rmtree('new//OEBPS//image/')
shutil.copytree('old//OEBPS//image/', 'new//OEBPS//image/')

# titelbild zu jpg umwandeln
os.system('convert new/OEBPS/1.png new/OEBPS/1.jpg')

os.remove('new//OEBPS//image//1.jpg')
os.remove('new//OEBPS//1.png')

shutil.move('new//OEBPS//1.jpg', 'new//OEBPS//image//1.jpg')    

'new//OEBPS//image//1.jpg'

In [12]:
# bilder dateiendungen ersetzen

for image in os.listdir('new//OEBPS//image/'):
    if fnmatch.fnmatch(image, '*.png'):
        
        # get filename withou extension
        name = os.path.splitext(image)[0]
        
        # replace in all files with the jpg version
        replaceAll(name+".jpg", image)
        
    if fnmatch.fnmatch(image, '*.jpg'):

        # get filename withou extension
        name = os.path.splitext(image)[0]

        # replace in all files with the png version
        replaceAll(name+".png", image)

In [13]:
find=r'<span class="Zitat_Anfuehrungszeichen _idGenCharOverride-\d+">„</span>'
replace='„'
replaceAll(find,replace, no_titel=True)

#anführungszeichen bei zitat nahc oben
find=r'<div id="_idContainer\d+" class="Zitat_Anfuehrungszeichen">\s*<p class="p_Zitat_Anfuehrungszeichen">„</p>\s*</div>'
replace=r''
replaceAll(find,replace, no_titel=True)

# zitat zu blockquote convertieren
find=r'<div id="(_idContainer\d\d\d)" class="Zitat">(.+?)</div>'
replace=r'<blockquote id="\1" class="Zitat">\2</blockquote>'
replaceAll(find,replace, flags=re.I | re.M | re.S, no_titel=True)

find='class="Zitat">'
replace = 'class="Zitat">\n\t\t\t\t\t<div class="Zitat_Anfuehrungszeichen">\n\t\t\t\t\t\t<p class="p_Zitat_Anfuehrungszeichen">„</p>\n\t\t\t\t\t</div>'
replaceAll(find,replace,reg=False, no_titel=True)

new//OEBPS/cover.xhtml
new//OEBPS/impressum.xhtml
new//OEBPS/titel_und_titelei.xhtml
new//OEBPS/toc.xhtml
new//OEBPS/cover.xhtml
new//OEBPS/impressum.xhtml
new//OEBPS/titel_und_titelei.xhtml
new//OEBPS/toc.xhtml
new//OEBPS/cover.xhtml
new//OEBPS/impressum.xhtml
new//OEBPS/titel_und_titelei.xhtml
new//OEBPS/toc.xhtml
new//OEBPS/cover.xhtml
new//OEBPS/impressum.xhtml
new//OEBPS/titel_und_titelei.xhtml
new//OEBPS/toc.xhtml


In [14]:
#add media.css
replaceAll('</title>','</title>\n\n\t\t<link href="css/media.css" rel="stylesheet" type="text/css" />', reg=False, no_titel= True)
#add covid.css

replaceAll('</title>','</title>\n\n\t\t<link href="css/COVID.css" rel="stylesheet" type="text/css" />', reg=False, no_titel= True)



new//OEBPS/cover.xhtml
new//OEBPS/impressum.xhtml
new//OEBPS/titel_und_titelei.xhtml
new//OEBPS/toc.xhtml
new//OEBPS/cover.xhtml
new//OEBPS/impressum.xhtml
new//OEBPS/titel_und_titelei.xhtml
new//OEBPS/toc.xhtml


In [15]:
# h2,h3,h4,h5 mit h1,h2,h2,h3 ersetzen
find = r'<h2(.+?)</h2>'
replace = r'<xxxh1\1</xxxh1>'
replaceAll(find,replace, flags=re.I | re.M | re.S, no_titel = True)

find = r'<h3(.+?)</h3>'
replace = r'<xxxh2\1</xxxh2>'
replaceAll(find,replace, flags=re.I | re.M | re.S, no_titel = True)

find = r'<h4(.+?)</h4>'
replace = r'<xxxh2\1</xxxh2>'
replaceAll(find,replace, flags=re.I | re.M | re.S, no_titel = True)

find = r'<h5(.+?)</h5>'
replace = r'<xxxh3\1</xxxh3>'
replaceAll(find,replace, flags=re.I | re.M | re.S, no_titel = True)

#und dann xxxh1 mit h1 ersetzen
find = 'xxxh1'
replace = 'h1'
replaceAll(find,replace, reg=False, no_titel = True)

find = 'xxxh2'
replace = 'h2'
replaceAll(find,replace, reg=False, no_titel = True)

find = 'xxxh3'
replace = 'h3'
replaceAll(find,replace, reg=False, no_titel = True)

new//OEBPS/cover.xhtml
new//OEBPS/impressum.xhtml
new//OEBPS/titel_und_titelei.xhtml
new//OEBPS/toc.xhtml
new//OEBPS/cover.xhtml
new//OEBPS/impressum.xhtml
new//OEBPS/titel_und_titelei.xhtml
new//OEBPS/toc.xhtml
new//OEBPS/cover.xhtml
new//OEBPS/impressum.xhtml
new//OEBPS/titel_und_titelei.xhtml
new//OEBPS/toc.xhtml
new//OEBPS/cover.xhtml
new//OEBPS/impressum.xhtml
new//OEBPS/titel_und_titelei.xhtml
new//OEBPS/toc.xhtml
new//OEBPS/cover.xhtml
new//OEBPS/impressum.xhtml
new//OEBPS/titel_und_titelei.xhtml
new//OEBPS/toc.xhtml
new//OEBPS/cover.xhtml
new//OEBPS/impressum.xhtml
new//OEBPS/titel_und_titelei.xhtml
new//OEBPS/toc.xhtml
new//OEBPS/cover.xhtml
new//OEBPS/impressum.xhtml
new//OEBPS/titel_und_titelei.xhtml
new//OEBPS/toc.xhtml


In [16]:

#fragezeichen tauschen + vereinfachen (vll dreht indesign undeterministisch manchmal um)
find = r'<div class="_idGenObjectLayout-\d+">\s*<div id="_idContainer\d+" class="Grafik_Fragezeichen _idGenObjectStyle-Disabled _idGenWritingDirection-1">\s*<div id="_idContainer\d+" class="Grafik_Fragezeichen">\s*<p class="p_STANDARD"><span class="Fragezeichen_rot _idGenCharOverride-\d+">\?</span></p>\s*</div>\s*<div id="_idContainer\d+" class="Grafik_Fragezeichen">\s*<p class="p_STANDARD"><span class="Fragezeichen_gruen _idGenCharOverride-\d+">\?</span></p>\s*</div>\s*</div>\s*</div>'
replace = r'<div class="fragezeichen-grafik-wrapper">\n\t\t\t\t<div class="fragezeichen-rot">?</div>\n\t\t\t\t<div class="fragezeichen-gruen">?</div>\n\t\t\t</div>'
replaceAll(find,replace)

find = r'<div class="_idGenObjectLayout-\d+">\s*<div id="_idContainer\d+" class="Grafik_Fragezeichen _idGenObjectStyle-Disabled _idGenWritingDirection-1">\s*<div id="_idContainer\d+" class="Grafik_Fragezeichen">\s*<p class="p_STANDARD"><span class="Fragezeichen_gruen _idGenCharOverride-\d+">\?</span></p>\s*</div>\s*<div id="_idContainer\d+" class="Grafik_Fragezeichen">\s*<p class="p_STANDARD"><span class="Fragezeichen_rot _idGenCharOverride-\d+">\?</span></p>\s*</div>\s*</div>\s*</div>'
replace = r'<div class="fragezeichen-grafik-wrapper">\n\t\t\t\t<div class="fragezeichen-rot">?</div>\n\t\t\t\t<div class="fragezeichen-gruen">?</div>\n\t\t\t</div>'
replaceAll(find,replace)

In [17]:
find = r'<div id="_idContainer\d+" class="Wellenlinie"><img class="_idGenObjectAttribute-\d+" src="image/wavy_line-ocker.png" alt="" /></div>'
replace =  r'<div class="wellenlinie-ocker"></div>'
replaceAll(find,replace)

# wellenlinie als hintergrund und repeatable
find = r'<div class="_idGenObjectLayout-\d+">\s*<div id="_idContainer\d+" class="Wellenlinie">\s*<img class="_idGenObjectAttribute-\d+" src="image/wavy_line.png" alt="" />\s*</div>\s*</div>'
replace = r'<div class="wellenlinie"></div>'
replaceAll(find,replace)

# manchmla auch ohne klasse
find = r'<div class="_idGenObjectLayout-\d+">\s*<div id="_idContainer\d+">\s*<img class="_idGenObjectAttribute-\d+" src="image/wavy_line.png" alt="" />\s*</div>\s*</div>'
replace = r'<div class="wellenlinie"></div>'
replaceAll(find,replace)

# ocker
find = r'<div class="_idGenObjectLayout-\d+">\s*<div id="_idContainer\d+" class="Wellenlinie">\s*<img class="_idGenObjectAttribute-\d+" src="image/wavy_line-ocker.png" alt="" />\s*</div>\s*</div>'
replace = r'<div class="wellenlinie-ocker"></div>'
replaceAll(find,replace)

# manchmla auch ohne klasse
find = r'<div class="_idGenObjectLayout-\d+">\s*<div id="_idContainer\d+">\s*<img class="_idGenObjectAttribute-\d+" src="image/wavy_line-ocker.png" alt="" />\s*</div>\s*</div>'
replace = r'<div class="wellenlinie"></div>'
replaceAll(find,replace)

In [18]:
# fragezeichen nach linie statt davor
find = r'<div class="fragezeichen-grafik-wrapper">\s*<div class="fragezeichen-rot">\?</div>\s*<div class="fragezeichen-gruen">\?</div>\s*</div>\s*<div class="wellenlinie"></div>'
replace = '<div class="wellenlinie"></div>\n\t\t\t<div class="fragezeichen-grafik-wrapper">\n\t\t\t\t<div class="fragezeichen-rot">?</div>\n\t\t\t\t<div class="fragezeichen-gruen">?</div>\n\t\t</div>'
replaceAll(find,replace)

In [19]:
# XXX nur diese auflage

# kap 1 fehler h5 absatz
find = '<h3 class="H5"><a id="_idTextAnchor029"></a><span class="CharOverride-9">Wie lange ein solcher'
replace = '<p class="p_STANDARD"><a id="_idTextAnchor029"></a><span class="CharOverride-9">Wie lange ein solcher'
replaceAll(find,replace, reg=False)

find = 'allenfalls lückenhaft sein). </span></h3>'
replace = 'allenfalls lückenhaft sein). </span></p>'
replaceAll(find,replace, reg=False)

In [20]:
# insert pagebreaks before anmerkungen
find = r'<div id="_idContainer\d\d\d" class="Basic-Text-Frame">\s*<h2 class="H4">'
replace = '<div class="Basic-Text-Frame" style="page-break-before: always;">\n\t\t\t<h2 class="H4">'
replaceAll(find,replace)

In [21]:
find = '_id'
replace = 'id'
replaceAll(find,replace, reg=False)

In [22]:
# copyright images nicht als bild

find=r'<h3 class="H5">\s*<img class="idGenObjectAttribute-9" src="image/2.png" alt=""\s*/>\s*</h3>'
replace='<p class="Bildunterschriften">© Holly (flickr.com/photos/37586339@N00/7075468) Lizenz: <a href="https://creativecommons.org/licenses/by/2.0/">CC BY 2.0</a></p>'
replaceAll(find,replace)


find = '<p class="p_STANDARD"><span class="Hyperlink"><img class="idGenObjectAttribute-6" src="image/3.png" alt="" /></span></p>'
replace = '<p class="Bildunterschriften"> © Ausschnitt aus dem Buch: <i>Gesundheit für Kinder – Kinderkrankheiten verhüten, erkennen, behandeln.</i> Von Herbert Renz-Polster, Nicole Menche und Arne Schäffler. 12. Auflage 2020, © Kösel Verlag</p>'
replaceAll(find,replace, reg=False)

In [23]:
# fix backling urls

find = r'endnote-(\d\d\d)-backlink-\d'
replace = r'endnote-\1-backlink'
replaceAll(find,replace)

find = r'endnote-(\d\d\d)-\d'
replace = r'endnote-\1'
replaceAll(find,replace)

In [24]:
# endnotemarker

find = r'<span class="Endnotenmarker idGenCharOverride-1"><span id="endnote-(\d\d\d)-backlink"><a class="idEndnoteLink idGenColorInherit" href="#endnote-\d\d\d">(\d+)</a></span></span>'
replace = r'<a epub:type="noteref" class="idEndnoteLink idGenColorInherit" href="#endnote-\1" id="endnote-\1-backlink"> [\2] </a>'
replaceAll(find,replace)

In [25]:
# footnotes

find = r'<p class="p_Anmerkungen"><span id="endnote-(\d\d\d)"><a class="idEndnoteAnchor idGenColorInherit" href="#endnote-\d\d\d-backlink">(\d+)</a>'
replace = r'<aside epub:type="rearnote" class="p_Anmerkungen" id="endnote-\1"><p><span><a class="idEndnoteAnchor idGenColorInherit" href="#endnote-\1-backlink"> [<span class="endnote-u">\2</span>] </a>'
replaceAll(find,replace)

In [26]:
find = r'</p>\s*<aside epub:type'
replace = '</p></aside>\n\t\t\t<aside epub:type'
replaceAll(find,replace)

In [27]:
for file in glob.glob("new//OEBPS/*.xhtml"):
        
        if file == "new//OEBPS/toc.xhtml" or file == "new//OEBPS/cover.xhtml" or file == "new//OEBPS/titel_und_titelei.xhtml" or file == "new//OEBPS/impressum.xhtml":
            continue
            
        content_new = ""
        with open (file, 'r' ) as f:
            content = f.read()
            
            content_new = re.sub(r'</p>\s*</div>\s*</body>', '</p></aside>\n\t\t</div>\n\t</body>', content, flags = re.I | re.M)

        with open (file, 'w' ) as f:
            f.write(content_new)

In [28]:
files = glob.glob("new//OEBPS/*.xhtml")
files.append('new//OEBPS/content.opf')
files.append('new//OEBPS/toc.ncx')

for file in files:
                  
        content_new = ""
        with open (file, 'r' ) as f:
            content = f.read()
            
            content = re.sub(r'Kapitel-(\d)-1.xhtml', r'Kapitel-\1.xhtml', content, flags = re.I | re.M)
            content = re.sub(r'<item id="Kapitel-\d-1" href="Kapitel-\d.xhtml" media-type="application/xhtml\+xml"/>', '', content, flags = re.I | re.M)

            
            find = r'Alles über Corona \(\d+\. Auflage\)'
            replace = 'Alles über Corona ('+auflage+'. Auflage)'
            content = re.sub(find, replace, content, flags = re.I | re.M)

            find = r'<dc:date>\d\d\d\d-\d\d-\d\dT\d\d:\d\d:\d\d\+\d\d:\d\d</dc:date>'
            replace = '<dc:date>'+date+'</dc:date>'
            content = re.sub(find, replace, content, flags = re.I | re.M)
            
            find = r'<itemref idref="Kapitel-\d+-\d"\s*/>'
            replace = ''
            content = re.sub(find, replace, content, flags = re.I | re.M)
        with open (file, 'w' ) as f:
            f.write(content)

In [29]:
find = r'\s*<div class="fragezeichen-grafik-wrapper">\s*<div class="fragezeichen-rot">\?</div>\s*<div class="fragezeichen-gruen">\?</div>\s*</div>\s*<h3 class="H5_Frage">Kann ich dieses eBook umsonst herunterladen\?</h3>.*?/corona-ebook</span></a></p>'
replaceAll(find,'', flags=re.I | re.M | re.S)

In [30]:
find = '<span class="Endnotenmarker CharOverride-3"><span id="endnote-001-backlink"><a class="idEndnoteLink idGenColorInherit" href="#endnote-001">1</a></span></span>'
replace = '<a epub:type="noteref" class="idEndnoteLink idGenColorInherit" href="#endnote-001" id="endnote-001-backlink"> [1] </a>'
replaceAll(find,replace, reg=False)

In [31]:
# im impressum auflage und datum updaten
find = r'Alles über Corona, \d+\. Auflage, \d\d\.\d\d\.\d\d\d\d\.'
replace = 'Alles über Corona, '+auflage+'. Auflage, '+datum
replaceAll(find,replace)
    
find = r'Herbert Renz-Polster: Alles über Corona\. Was Du wissen musst\. Was du tun kannst. \d+\. Auflage\.'
replace = 'Herbert Renz-Polster: Alles über Corona. Was Du wissen musst. Was du tun kannst. '+auflage+'. Auflage.'
replaceAll(find,replace)

In [32]:
# schwäne kap 4 illu nicht vorhanden
find = 'Illustration Kapitel 4 (</span><a href="Kapitel-4-1.xhtml#idTextAnchor059"><span class="CharOverride-10">Seite </span><span class="CharOverride-10">73</span></a><span class="CharOverride-10">): Marc Robitzky, aus Renz/Robitzky: Die wilden Schwäne. Zweisprachiges Kinderbuch nach einem Märchen von Hans Christian Andersen. www.sefa-bilingual.com.'
replace = ''
replaceAll(find, replace, reg=False)

# kap 4 anfang deswegen verschoben
find = 'Kapitel-4.xhtml#idTextAnchor112'
replace = 'Kapitel-4.xhtml#idTextAnchor056'
replaceAll(find, replace, reg=False)

# eliah reference einfügen für impressum

find = 'Kapitel-4.xhtml#idTextAnchor112'
replace = 'Kapitel-4.xhtml#idTextAnchor056'
replaceAll(find, replace, reg=False)

find = 'Kapitel-4.xhtml#idContainer169'
replace = 'Kapitel-4.xhtml#idAnchor001'
replaceAll(find, replace, reg=False)

find = '<img class="idGenObjectAttribute-4" src="image/eliah.jpg" alt="" />'
replace = '<a id="idAnchor001"></a><img class="idGenObjectAttribute-4" src="image/eliah.jpg" alt="" />'
replaceAll(find, replace, reg=False)

# fix toc
find = '#toc_1'
f = ['new//OEBPS/toc.ncx', 'new//OEBPS/toc.xhtml']
replaceAll(find, '', reg=False, files=f)

In [33]:
# komische zeichen

find = '&#9;'
replace = ' '
replaceAll(find,replace, reg=False)

find = '&#160;'
replace = ' '
replaceAll(find,replace, reg=False)

find = ' '
replace = ' '
replaceAll(find,replace, reg=False)

In [34]:
# bad nyt links
find = '?action=click&amp;module=Opinion&amp;pgtype=Homepage'
replace = ''
replaceAll(find,replace, reg=False)

find = '?action=click&amp;module=Top%20Stories&amp;pgtype=Homepage'
replace = ''
replaceAll(find,replace, reg=False)


find = 'http://www.gbe-bund.de/oowa921-install/servlet/oowa/aw92/dboowasys921.xwdevkit/xwd_init?gbe.isgbetol/xs_start_neu/&amp;p_aid=3&amp;p_aid=46516668&amp;nummer=524&amp;p_sprache=D&amp;p_indsp=-&amp;p_aid=41622080'
replace = '<a href="http://www.gbe-bund.de/oowa921-install/servlet/oowa/aw92/dboowasys921.xwdevkit/xwd_init?gbe.isgbetol/xs_start_neu/&amp;p_aid=3&amp;p_aid=46516668&amp;nummer=524&amp;p_sprache=D&amp;p_indsp=-&amp;p_aid=41622080">www.gbe-bund.de/oowa921-install/servlet/oowa/aw92/dboowasys921.xwdevkit/xwd_init?gbe.isgbetol/xs_start_neu/&amp;p_aid=3&amp;p_aid=46516668&amp;nummer=524&amp;p_sprache=D&amp;p_indsp=-&amp;p_aid=41622080</a>'
replaceAll(find,replace, reg=False)

In [35]:
# replace all urls with that dont start with "
reg1 = r'(?<!\")(https?:\/\/(?:(?:https?|ftp|file):\/\/|www\.|ftp\.)(?:\([-A-Z0-9+&@#\/%=~_|$?!:,.]*\)|[-A-Z0-9+&@#\/%=~_|$?!:,.])*(?:\([-A-Z0-9+&@#\/%=~_|$?!:,.]*\)|[A-Z0-9+&@#\/%=~_|$]))'
find = r'(?<!\")((http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?)'   
replace = r'<a href="\1">\1</a>'
replaceAll(find,replace)

In [36]:
# fix broken youtube link

find = '<a href="https://youtu.be/bsffWx5Kx_I?list=PL1G2CRcxbVQOnhvr1eG7ToK_wMvmHQFRR"><span class="Hyperlink"><a href="https://youtu.be/">https://youtu.be/</a></span></a><span class="Hyperlink">…</span>'
replace = '<a href="https://youtu.be/bsffWx5Kx_I?list=PL1G2CRcxbVQOnhvr1eG7ToK_wMvmHQFRR"><span class="Hyperlink">https://youtu.be/…</span></a>'
replaceAll(find,replace, reg = False)

In [37]:
# add width autoclass to some images

images = ['image/tabelle_symptome.jpg',
          'image/GfK11_Kap_08_159.jpg',
          'image/tweet_lauterbach.jpg',
          'image/Corona_Sterblichkeit_NYC.jpg',
          'image/Unterschrift_Re-Po.jpg',
          'image/KV_blogbeitr_ge.jpg']

for image in images:
    find=r'img class="(.*?)" src="'+image+'"'
    replace=r'img class="\1 width-auto" src="'+image+'"'
    replaceAll(find,replace)

In [38]:
# delet image

find = r'<div class="idGenObjectLayout-1">\s*<div id="idContainer218" class="Standard_bild">\s*<img class="idGenObjectAttribute-1" src="image/vitamin_c_cc0.jpg" alt="" />\s*</div>\s*</div>'
replace = ''
replaceAll(find,replace)

In [39]:
p = subprocess.Popen(['zip','-rX', "../converted_"+new_epub,'mimetype','META-INF/','OEBPS/'], cwd='new/')
p.wait()

0

In [40]:
# cleanup
shutil.rmtree('new/')
shutil.rmtree('old/')